In [ ]:
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist

In [ ]:
file = "data/war_crimes_list_2018-06-04.xlsx"

In [ ]:
df = pd.read_excel(file).fillna('False')

In [ ]:
relation_by_crime_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='crime'
)
relation_type_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='person_warcase_relation_type'
)

In [ ]:
issues = []
for i, row in df.iterrows():
    try:
        pers = Person.objects.get(dog_tag=row['Dienstnummer'])
    except ObjectDoesNotExist:
        pers = None
        issues.append(row)
    if row['Nummer'] != 'False' and pers:
        case, _ = WarCrimeCase.objects.get_or_create(
            legacy_id=row['Nummer']
        )
        pw, _ = PersonWarCrimeCase.objects.get_or_create(
            legacy_id=row['ID']
        )
        pw.related_person = pers
        pw.related_case = case
        pw.save()
        if row['Zeitpunkt'] != 'False':
            start_date = None
            try:
                start_date = pd.to_datetime(row['Zeitpunkt'], dayfirst=True)
            except:
                start_date = None
            if start_date:
                pw.start_date = start_date
        
        if row['Einordung (Opfer/Zeuge)'] != 'False':
            relation_type, _ = SkosConcept.objects.get_or_create(
                pref_label=row['Einordung (Opfer/Zeuge)']
            )
            relation_type.scheme.set([relation_type_scheme])
            pw.relation_type = relation_type
        
        if row['Art Verbrechen'] != 'False':
            relation_by_crime, _ = SkosConcept.objects.get_or_create(
                pref_label=row['Art Verbrechen']
            )
            relation_by_crime.scheme.set([relation_by_crime_scheme])
            pw.relation_by_crime = relation_by_crime
        
        if row['Anmerkung'] != 'False':
            pw.comment = row['Anmerkung']
        signatur = None
        if row['Cases Tried'] != 'False' and row['Cases Not Tried']:
            signatur = "{}|{}".format(row['Cases Tried'], row['Cases Not Tried'])
        elif row['Cases Tried'] != 'False':
            signatur = row['Cases Tried']
        else:
            signatur = row['Cases Not Tried']
        if signatur:
            case.signatur = signatur 
            case.save()
        pw.save()
        

In [ ]:
len(issues)